<a href="https://colab.research.google.com/github/Apoorvkhanna2/energypredictor/blob/main/Energy_Consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy matplotlib scikit-learn gradio plotly
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle
import gradio as gr
import datetime

# ========== ENERGY SOURCE DATA ==========
energy_sources = {
    "Petroleum": {
        "description": "Fossil fuel formed from ancient organic matter",
        "global_reserves": "244 billion tonnes (2022)",
        "depletion_time": "About 50 years at current usage",
        "sustainability": "Low",
        "co2_emissions": "73.2 kg CO2 per million BTU",
        "alternatives": ["Solar", "Wind", "Biofuels"],
        "efficiency": "30-40% in power generation"
    },
    "Natural Gas": {
        "description": "Fossil fuel primarily methane",
        "global_reserves": "188 trillion cubic meters (2022)",
        "depletion_time": "About 52 years at current usage",
        "sustainability": "Medium (cleanest fossil fuel)",
        "co2_emissions": "53.1 kg CO2 per million BTU",
        "alternatives": ["Biogas", "Hydrogen", "Geothermal"],
        "efficiency": "40-60% in power generation"
    },
    "Coal": {
        "description": "Combustible black sedimentary rock",
        "global_reserves": "1.07 trillion tonnes (2022)",
        "depletion_time": "About 132 years at current usage",
        "sustainability": "Very low",
        "co2_emissions": "95.3 kg CO2 per million BTU",
        "alternatives": ["Nuclear", "Solar", "Wind"],
        "efficiency": "33-40% in power generation"
    },
    "Renewables": {
        "description": "Energy from solar, wind, hydro, etc.",
        "global_reserves": "Practically unlimited",
        "depletion_time": "N/A",
        "sustainability": "High",
        "co2_emissions": "Minimal during operation",
        "alternatives": ["Energy storage solutions"],
        "efficiency": "Varies (15-90% depending on technology)"
    }
}

# ========== DATA PROCESSING ==========
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    df['Building Type'] = df['Building Type'].map({'Residential': 0, 'Commercial': 1})
    df['Day of Week'] = df['Day of Week'].map({'Weekday': 0, 'Weekend': 1})

    # Add some dummy sustainability metrics
    df['Carbon Footprint'] = df['Energy Consumption'] * np.random.uniform(0.8, 1.2, len(df))
    df['Potential Savings'] = df['Energy Consumption'] * np.random.uniform(0.1, 0.3, len(df))

    X = df[['Building Type', 'Square Footage', 'Number of Occupants',
            'Appliances Used', 'Average Temperature', 'Day of Week']]
    y = df['Energy Consumption']

    return df, X, y

# ========== MODEL TRAINING ==========
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Model Performance: Mean Squared Error = {mse:.2f}")

    with open("energy_model.pkl", "wb") as file:
        pickle.dump(model, file)
    print("Model saved as 'energy_model.pkl'")

    return model

# Load dataset and model
file_path = "/content/train_energy_data.csv"
try:
    with open("energy_model.pkl", "rb") as file:
        model = pickle.load(file)
    print("Loaded existing model.")
    df, X, y = load_dataset(file_path)
except FileNotFoundError:
    print("Training new model...")
    df, X, y = load_dataset(file_path)
    model = train_model(X, y)

# ========== SUSTAINABILITY FUNCTIONS ==========
def get_sustainability_tips(prediction, avg_consumption, building_type):
    difference = prediction - avg_consumption
    percentage_diff = (difference / avg_consumption) * 100

    base_tips = [
        "Switch to LED lighting which uses 75% less energy",
        "Install smart thermostats to optimize heating/cooling",
        "Seal windows and doors to prevent energy leaks",
        "Use natural ventilation when possible",
        "Unplug devices when not in use to prevent phantom loads"
    ]

    commercial_tips = [
        "Implement an energy management system",
        "Conduct regular energy audits",
        "Upgrade to Energy Star certified equipment",
        "Optimize HVAC schedules based on occupancy",
        "Consider rooftop solar panels"
    ] if building_type == "Commercial" else [
        "Wash clothes in cold water when possible",
        "Air dry clothes instead of using a dryer",
        "Install low-flow showerheads to reduce water heating needs",
        "Use microwave instead of oven for small meals",
        "Plant shade trees near windows to reduce cooling needs"
    ]

    if percentage_diff > 20:
        severity = "high"
        additional_tips = [
            "Consider a professional energy audit",
            "Evaluate insulation in walls and attic",
            "Upgrade to energy-efficient appliances immediately",
            "Install solar panels to offset high usage",
            "Implement strict energy-saving policies"
        ]
    elif percentage_diff > 10:
        severity = "medium"
        additional_tips = [
            "Replace old appliances with Energy Star models",
            "Install programmable thermostats",
            "Use power strips for electronics to easily turn them off",
            "Set computers to energy-saving mode",
            "Lower water heater temperature to 120°F"
        ]
    else:
        severity = "low"
        additional_tips = [
            "Maintain your current good practices",
            "Monitor usage patterns for further optimization",
            "Consider small upgrades when appliances need replacement",
            "Educate occupants about energy conservation",
            "Regularly maintain HVAC systems"
        ]

    return {
        "severity": severity,
        "percentage_diff": abs(round(percentage_diff, 1)),
        "base_tips": base_tips,
        "specific_tips": commercial_tips,
        "additional_tips": additional_tips
    }

def generate_energy_source_html():
    html_content = """
    <div style="font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto;">
        <h2 style="color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 5px;">Global Energy Source Information</h2>
    """

    for source, data in energy_sources.items():
        color = "#e74c3c" if data["sustainability"] == "Low" else "#f39c12" if data["sustainability"] == "Medium" else "#2ecc71"

        html_content += f"""
        <div style="background-color: #f9f9f9; padding: 15px; margin-bottom: 20px; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <h3 style="color: #2980b9; margin-top: 0;">{source}</h3>
            <p><strong>Description:</strong> {data["description"]}</p>
            <p><strong>Global Reserves:</strong> {data["global_reserves"]}</p>
            <p><strong>Estimated Depletion Time:</strong> {data["depletion_time"]}</p>
            <p><strong>Sustainability:</strong> <span style="color: {color}; font-weight: bold;">{data["sustainability"]}</span></p>
            <p><strong>CO2 Emissions:</strong> {data["co2_emissions"]}</p>
            <p><strong>Recommended Alternatives:</strong> {", ".join(data["alternatives"])}</p>
            <p><strong>Typical Efficiency:</strong> {data["efficiency"]}</p>
        </div>
        """

    html_content += """
        <div style="margin-top: 30px; padding: 15px; background-color: #e8f4f8; border-radius: 5px;">
            <h3 style="color: #16a085; margin-top: 0;">Sustainability Recommendations</h3>
            <p>To ensure sustainable energy future, consider:</p>
            <ul>
                <li>Transitioning to renewable energy sources</li>
                <li>Improving energy efficiency in all sectors</li>
                <li>Investing in energy storage technologies</li>
                <li>Supporting policies that promote clean energy</li>
                <li>Educating communities about conservation</li>
            </ul>
        </div>
    </div>
    """

    return html_content

def generate_report(building_type, sq_ft, occupants, appliances, temp, day, prediction, avg, tips):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Generate PDF-like HTML content
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Energy Consumption Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; margin: 0; padding: 20px; color: #333; }}
            .header {{ border-bottom: 2px solid #3498db; padding-bottom: 10px; margin-bottom: 20px; }}
            .section {{ margin-bottom: 25px; }}
            h1 {{ color: #2c3e50; }}
            h2 {{ color: #2980b9; border-bottom: 1px solid #eee; padding-bottom: 5px; }}
            .highlight {{ background-color: #f8f9fa; padding: 15px; border-radius: 5px; }}
            .tip {{ background-color: #e8f4f8; padding: 10px; margin: 10px 0; border-left: 4px solid #3498db; }}
            .footer {{ margin-top: 30px; font-size: 0.9em; color: #7f8c8d; text-align: center; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>Energy Consumption Analysis Report</h1>
            <p>Generated on: {timestamp}</p>
        </div>

        <div class="section">
            <h2>Input Parameters</h2>
            <ul>
                <li><strong>Building Type:</strong> {building_type}</li>
                <li><strong>Square Footage:</strong> {sq_ft} sq. ft.</li>
                <li><strong>Number of Occupants:</strong> {occupants}</li>
                <li><strong>Appliances Used:</strong> {appliances}</li>
                <li><strong>Average Temperature:</strong> {temp}°C</li>
                <li><strong>Day Type:</strong> {day}</li>
            </ul>
        </div>

        <div class="section highlight">
            <h2>Consumption Analysis</h2>
            <p><strong>Predicted Energy Consumption:</strong> {prediction:.2f} kWh</p>
            <p><strong>Average Consumption (Benchmark):</strong> {avg:.2f} kWh</p>
            <p><strong>Difference:</strong> {prediction - avg:.2f} kWh ({tips['percentage_diff']}%)</p>
            <p><strong>Severity Level:</strong> {tips['severity'].upper()}</p>
        </div>

        <div class="section">
            <h2>Recommendations</h2>
            <h3>General Energy Saving Tips</h3>
            {"".join([f'<div class="tip">{tip}</div>' for tip in tips['base_tips']])}

            <h3>{building_type}-Specific Tips</h3>
            {"".join([f'<div class="tip">{tip}</div>' for tip in tips['specific_tips']])}

            <h3>Additional Recommendations Based on Your Usage</h3>
            {"".join([f'<div class="tip">{tip}</div>' for tip in tips['additional_tips']])}
        </div>

        <div class="section">
            <h2>Sustainability Information</h2>
            <p>Your current carbon footprint for this prediction: <strong>{prediction * 0.85:.2f} kg CO2</strong></p>
            <p>Potential savings if recommendations are implemented: <strong>{prediction * 0.15:.2f} kWh</strong></p>
        </div>

        <div class="footer">
            <p>This report was generated by the Sustainable Energy Consumption Tracking System</p>
        </div>
    </body>
    </html>
    """

    # Save HTML to a file
    report_filename = f"/tmp/energy_report_{timestamp.replace(' ', '_').replace(':', '-')}.html"
    with open(report_filename, "w") as file:
        file.write(html_content)

    return report_filename

# ========== MAIN PREDICTION FUNCTION ==========
def predict_energy_consumption(building_type, square_footage, occupants, appliances, temperature, day_of_week):
    # Map input values
    building_type_num = 1 if building_type == "Commercial" else 0
    day_of_week_num = 1 if day_of_week == "Weekend" else 0

    # Prepare the input for the model
    features = np.array([[building_type_num, square_footage, occupants, appliances, temperature, day_of_week_num]])
    prediction = model.predict(features)[0]

    # Get average consumption
    avg_consumption = df['Energy Consumption'].mean()

    # Get sustainability tips
    tips = get_sustainability_tips(prediction, avg_consumption, building_type)

    # Generate visualizations
    fig1, ax1 = plt.subplots(figsize=(8, 5))
    categories = ['Your Prediction', 'Average']
    values = [prediction, avg_consumption]
    colors = ['#e74c3c' if prediction > avg_consumption else '#2ecc71', '#3498db']
    ax1.bar(categories, values, color=colors)
    ax1.set_ylabel('Energy Consumption (kWh)')
    ax1.set_title('Energy Consumption Comparison')
    ax1.grid(axis='y', linestyle='--', alpha=0.7)
    for i, v in enumerate(values):
        ax1.text(i, v + 0.5, f"{v:.2f}", ha='center')
    plt.tight_layout()
    plt.savefig('/tmp/consumption_comparison.png')
    plt.close()

    # Generate pie chart of potential savings
    potential_savings = prediction * 0.2  # Assume 20% potential savings
    fig2, ax2 = plt.subplots(figsize=(6, 6))
    ax2.pie([prediction - potential_savings, potential_savings],
            labels=['Current Usage', 'Potential Savings'],
            colors=['#3498db', '#2ecc71'],
            autopct='%1.1f%%',
            startangle=90)
    ax2.set_title('Potential Energy Savings')
    plt.tight_layout()
    plt.savefig('/tmp/potential_savings.png')
    plt.close()

    # Generate report
    report_filename = generate_report(
        building_type, square_footage, occupants, appliances,
        temperature, day_of_week, prediction, avg_consumption, tips
    )

    # Format tips for display
    formatted_tips = "===== GENERAL TIPS =====\n" + "\n".join(tips['base_tips']) + "\n\n"
    formatted_tips += f"===== {building_type.upper()}-SPECIFIC TIPS =====\n" + "\n".join(tips['specific_tips']) + "\n\n"
    formatted_tips += "===== ADDITIONAL RECOMMENDATIONS =====\n" + "\n".join(tips['additional_tips'])

    # Calculate carbon footprint
    carbon_footprint = prediction * 0.85  # Simplified conversion factor

    return (
        f"Predicted Energy Consumption: {prediction:.2f} kWh\n"
        f"Average Consumption: {avg_consumption:.2f} kWh\n"
        f"Difference: {prediction - avg_consumption:.2f} kWh ({tips['percentage_diff']}% {'higher' if prediction > avg_consumption else 'lower'})\n"
        f"Carbon Footprint: {carbon_footprint:.2f} kg CO2\n"
        f"Potential Savings: {potential_savings:.2f} kWh",
        '/tmp/consumption_comparison.png',
        '/tmp/potential_savings.png',
        formatted_tips,
        report_filename
    )

# ========== GRADIO INTERFACE ==========
with gr.Blocks(title="Sustainable Energy Tracker", theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    # 🌱 Sustainable Energy Consumption Tracking System
    ### Monitor, Analyze, and Optimize Your Energy Usage
    """)

    with gr.Tabs():
        with gr.TabItem("📊 Energy Prediction"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Enter Your Building Details")
                    building_type = gr.Radio(["Residential", "Commercial"], label="Building Type", value="Residential")
                    square_footage = gr.Number(label="Square Footage (sq. ft.)", minimum=100, value=1500)
                    occupants = gr.Number(label="Number of Occupants", minimum=1, value=3)
                    appliances = gr.Number(label="Number of Appliances Used", minimum=1, value=5)
                    temperature = gr.Slider(-30, 50, step=1, label="Average Temperature (°C)", value=20)
                    day_of_week = gr.Radio(["Weekday", "Weekend"], label="Day of Week", value="Weekday")
                    predict_button = gr.Button("Predict Energy Consumption", variant="primary")

                with gr.Column():
                    gr.Markdown("### Prediction Results")
                    results_output = gr.Textbox(label="Analysis Results", interactive=False)
                    comparison_plot = gr.Image(label="Consumption Comparison")
                    savings_plot = gr.Image(label="Potential Savings")

            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Personalized Recommendations")
                    tips_output = gr.Textbox(label="Energy Saving Tips", lines=10, interactive=False)

                with gr.Column():
                    gr.Markdown("### Report Generation")
                    report_download = gr.File(label="Download Full Report")
                    gr.Markdown("""
                    **Your report includes:**
                    - Detailed consumption analysis
                    - Personalized recommendations
                    - Carbon footprint calculation
                    - Long-term sustainability tips
                    """)

        with gr.TabItem("🌍 Sustainability Hub"):
            gr.Markdown("## Global Energy Sources Information")
            energy_html = gr.HTML(generate_energy_source_html())

            gr.Markdown("## Sustainable Development Goals (SDGs) for Energy")
            with gr.Row():
                with gr.Column():
                    gr.Markdown("""
                    **SDG 7: Affordable and Clean Energy**
                    - Ensure access to affordable, reliable, sustainable energy for all
                    - Increase share of renewables in global energy mix
                    - Double global rate of improvement in energy efficiency
                    """)
                with gr.Column():
                    gr.Markdown("""
                    **Key Targets for 2030:**
                    - Universal access to electricity
                    - Substantially increase renewable energy share
                    - Enhance international cooperation for clean energy research
                    - Expand infrastructure for modern energy services
                    """)

            gr.Markdown("## Carbon Footprint Calculator")
            with gr.Row():
                electricity = gr.Number(label="Monthly Electricity Usage (kWh)", value=300)
                gas = gr.Number(label="Monthly Gas Usage (therms)", value=50)
                fuel = gr.Number(label="Monthly Vehicle Fuel (gallons)", value=25)
            calc_footprint = gr.Button("Calculate Carbon Footprint")
            footprint_result = gr.Markdown()

            @calc_footprint.click(inputs=[electricity, gas, fuel], outputs=footprint_result)
            def calculate_footprint(elec, gas, fuel):
                # Simplified conversion factors
                elec_emissions = elec * 0.85  # kg CO2/kWh (US average)
                gas_emissions = gas * 5.3     # kg CO2/therm
                fuel_emissions = fuel * 8.89  # kg CO2/gallon
                total = elec_emissions + gas_emissions + fuel_emissions

                return f"""
                ### Your Estimated Carbon Footprint
                - **Electricity:** {elec_emissions:.1f} kg CO2/month
                - **Gas:** {gas_emissions:.1f} kg CO2/month
                - **Vehicle Fuel:** {fuel_emissions:.1f} kg CO2/month
                - **TOTAL:** {total:.1f} kg CO2/month ({total*12:.1f} kg annually)

                *The average INDIA household produces about 7,500 kg CO2 annually from energy use.*
                """

    # Connect prediction button
    predict_button.click(
        predict_energy_consumption,
        inputs=[building_type, square_footage, occupants, appliances, temperature, day_of_week],
        outputs=[results_output, comparison_plot, savings_plot, tips_output, report_download]
    )

# Launch the app
app.launch(share=True)


Loaded existing model.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://662f774ad86db8f35c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
